In [1]:
import os
import sys
import json
from iconic_tools.langchain import InstructSonnet, InstructOpus3, InstructGPT4, InstructGeminiPro, InstructGPT35
from langchain_core.prompts import ChatPromptTemplate
from benchmark import dataset_utils

/Users/murrayshanahan/iconic_dir/digital_actor_benchmark/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CONSTANTS AND INITIALISATION

PATH = os.path.abspath(os.getcwd())

# DIALOGUE_MODEL = InstructGeminiPro(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGeminiPro(temperature=1.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructGPT4(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGPT4(temperature=1.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructSonnet(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructSonnet(temperature=1.0, max_tokens=3000)

# GAME = "amadeus"
# SCENE = "constanze"
# ACTORS = ["Mozart", "Constanze", "Schikaneder"]
# GENRE = "drama"
# SCENE_IDX = 124

# GAME = "bound"
# SCENE = "coffee"
# ACTORS = ["Violet", "Corky"]
# GENRE = "drama"
# SCENE_IDX = 2

# GAME = "cedar_rapids"
# SCENE = "car"
# ACTORS = ["Tim", "Ronald", "Dean", "Joan"]
# GENRE = "comedy"
# SCENE_IDX = 116

GAME = "troy"
SCENE = "achilles"
ACTORS = ["Achilles", "Odysseus"]
GENRE = "action"
SCENE_IDX = 0

RED = "\033[91m"
GREEN = "\033[92m"
BLUE = "\033[94m"
YELLOW = "\033[93m"
WHITE = "\033[0m"

In [3]:
# UTILITIES


def print_header(model_name):
    print(WHITE + "Game or movie: {}".format(GAME))
    print("Scene name: {}".format(SCENE))
    print("Dialogue model: {}".format(model_name))
    print()


def load_prompt(filename):
    with open(PATH + f"/prompts/{filename}") as f:
        return f.read()


def write_transcript(dialogue, filename):
    with open(PATH + f"/transcripts/{filename}", "w") as f:
        f.write(dialogue)


def list_to_conjunction(L):
    """Takes a list strings and returns a string with every element in the list separated by commas."""
    if L == "":
        return ""
    elif len(L) == 1:
        return L[0]
    elif len(L) == 2:
        return f"{L[0]} and {L[1]}"
    else:
        return ", ".join(L[:-1]) + f", and {L[-1]}"


def list_to_string(L):
    """Takes a list of strings and returns a string consisting of every element in the list separated by a newline."""
    return "\n".join(L)


def split_text(text):
    # Split the text by double newlines
    paragraphs = text.split('\n\n')
    # Remove leading or trailing whitespacen  and remove empty paragraphs
    paragraphs = [p.strip() for p in paragraphs]
    paragraphs = [p for p in paragraphs if p]
    return paragraphs


def read_queries(filename):
    queries = split_text(load_prompt(filename))
    queries = [s.strip() for s in queries if not s.strip().startswith('#')]
    return queries

In [4]:
# PROMPT TEMPLATES AND INSTRUCTION PROMPTS


dialogue_instruction_prefix = """
You are going to generate one line of dialogue for a scene in the middle of a computer game.
"""

preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
Here is a description of the scene in question. {scene_description}{scene_supplement}\n
The characters in the dialogue are {actors}.
"""

instruction_template = """
{preamble}
Here is the dialogue so far\n\n
{dialogue}
{instruction_suffix}
"""

speech_template = '[{actor}]: {speech}\n'

dialogue_instruction_suffix = """
Give me the next line in the dialogue in the same format. Don't provide stage directions, just the actor's words.\n
"""

query_preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
"""

query_instruction_prefix = """
You are going to answer a single question about the current state of the dialogue in a scene in the middle of a computer game.
"""

query_instruction_suffix_template = """
Now consider the following statement about this dialogue. {statement} Is this statement true or false? Answer with a single word, true or false.
"""

naive_dialogue_prompt = """
Given the following dialogue, predict the next line in that dialogue. Respond with the next line only. Use the same format as the dialogue so far. Don't provide stage directions, just the actor's words. Here's the dialogue until now, along with the contextual information:
"""

In [5]:
# BUILDING DIALOGUES


def prompt_llm(prompt, model):
    # print(prompt)
    # print()
    prompt = ChatPromptTemplate.from_template(template=prompt)
    chain = prompt | model
    return chain


def load_handmade_prompts(supplement_version=-1):
    """Loads a set of handmade prompts for the current game and scene.

    back_story: The backdrop to the whole game.
    scene_description: A description of this particular mini-scene. Should state the goals of the scene.
    scene_supplement: Extra text describing the scene. Can be used to simulate adversarial players, for example.
    actors: A list of the names of the actors involved in generating dialogue.
    opening_speech: The first words spoken by actor1. (These have to be scripted.)
    player: If True then the user is one of the players, otherwise both players are LLMs.
    end_queries: A list of natural language questions; the scene is terminated if all answers are yes.
    max_turns: The scene will terminate when this many turns have been taken whether or not goals have been reached.
    """

    prompt_method = "handmade"
    back_story = load_prompt(GAME + "/back_story.txt")
    scene_description = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_scene_description_" + str(prompt_method) + ".txt")
    if supplement_version == -1:  # no supplementary scene text
        scene_supplement = ""
    else:
        scene_supplement = "\n\n" + load_prompt(
            GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_scene_supplement_" + str(prompt_method) + ".txt")
    opening_speech = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_opening_speech.txt")
    queries = read_queries(GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_queries.txt")
    return (back_story, scene_description, scene_supplement, opening_speech, queries)


def sim_mini_scene_naive(supplement_version, player, max_turns):
    """Generates dialogue for a mini-scene using naive prompting.
    """
    
    dialogue_model = DIALOGUE_MODEL
    actors = ACTORS

    opening_speech = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_opening_speech.txt")

    (dialogue, dialogue_state) = dataset_utils.load_scene(GENRE, GAME, SCENE_IDX)

    facts = list_to_string(dialogue.facts)
    comm_style = list_to_string(dialogue.comm_style)
    goals = list_to_string(dialogue.goals)
    
    query_preamble = preamble_template.format(
        instruction_prefix=query_instruction_prefix,
        back_story="",
        scene_description="",
        scene_supplement="",
        actors=list_to_conjunction(actors))
    
    queries = read_queries(GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_queries.txt")
    
    if player:
        speech = input()
        response = speech_template.format(actor=actors[0], speech=speech)
    else:
        response = speech_template.format(actor=actors[0], speech=opening_speech)

    dialogue = response + "\n"

    print(GREEN + response)

    turn = 1
    success = False

    while turn < max_turns and not success:
        
        if player and (turn % 2 == 0):  # user is playing actor 1
            speech = input()
            response = speech_template.format(actor=actors[0], speech=speech)
        else:  # both actors are played by the LLM
            prompt = (naive_dialogue_prompt + "\n" + facts + "\n" + comm_style + "\n" + goals +
                      "\n\n" + ACTORS[0] + ": " + dialogue)
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({}) + "\n"

        dialogue += response

        print(GREEN + response)

        # Have the conditions for ending the scene been met?
        fails = 0
        for statement in queries:                
            instruction = query_instruction_suffix_template.format(statement=statement)
            prompt = instruction_template.format(
                preamble=query_preamble, dialogue=dialogue,
                instruction_suffix=instruction)
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({})
            if response[0:4] != "True" and response[0:3] != "true":
                fails += 1
        success = (fails == 0)

        turn += 1
    
    return dialogue


def sim_mini_scene_handmade(supplement_version, player, max_turns):
    """Generates dialogue for a mini-scene using handmade prompts.
    """

    dialogue_model = DIALOGUE_MODEL
    actors = ACTORS

    (back_story, scene_description, scene_supplement,
     opening_speech, queries) = load_handmade_prompts(supplement_version)
    
    dialogue_preamble = preamble_template.format(
        instruction_prefix=dialogue_instruction_prefix,
        back_story=back_story,
        scene_description=scene_description,
        scene_supplement=scene_supplement,
        actors=list_to_conjunction(actors))
    
    query_preamble = preamble_template.format(
        instruction_prefix=query_instruction_prefix,
        back_story=back_story,
        scene_description="",
        scene_supplement="",
        actors=list_to_conjunction(actors))
    
    if player:
        speech = input()
        response = speech_template.format(actor=actors[0], speech=speech)
    else:
        response = speech_template.format(actor=actors[0], speech=opening_speech)

    dialogue = response + "\n"

    print(GREEN + response)

    turn = 1
    success = False

    while turn < max_turns and not success:
        
        if player and (turn % 2 == 0):  # user is playing actor 1
            speech = input()
            response = speech_template.format(actor=actors[0], speech=speech)
        else:  # both actors are played by the LLM
            prompt = instruction_template.format(
                preamble=dialogue_preamble, dialogue=dialogue,
                instruction_suffix=dialogue_instruction_suffix)
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({}) + "\n"

        dialogue += response

        print(GREEN + response)

        # Have the conditions for ending the scene been met?
        fails = 0
        for statement in queries:                
            instruction = query_instruction_suffix_template.format(statement=statement)
            prompt = instruction_template.format(
                preamble=query_preamble, dialogue=dialogue,
                instruction_suffix=instruction)
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({})
            if response[0:4] != "True" and response[0:3] != "true":
                fails += 1
        success = (fails == 0)

        turn += 1
    
    return dialogue


def sim_mini_scene(prompt_method, supplement_version, player=False, max_turns=10):
    match prompt_method:
        case "naive":
            return sim_mini_scene_naive(supplement_version, player, max_turns)
        case "handmade":
            return sim_mini_scene_handmade(supplement_version, player, max_turns)

In [ ]:
# GENERATE AND SAVE DIALOGUES

player = False
supplement_version = -1
n_runs = 3
models = [InstructSonnet(temperature=1.0, max_tokens=3000),
          InstructGPT4(temperature=1.0, max_tokens=3000),
          InstructGeminiPro(temperature=1.0, max_tokens=3000)]
model_names = ["Sonnet", "GPT4", "GeminiPro"]
prompt_methods = ["naive", "handmade"]

for i in range(len(models)):
    DIALOGUE_MODEL = models[i]
    model_name = model_names[i]
    for prompt_method in prompt_methods:
        print_header(model_name)
        print(WHITE + "Prompt method: {}".format(prompt_method))
        print()
        for run_no in range(n_runs):
            print(WHITE + "Run {}".format(run_no))
            print()
            dialogue = sim_mini_scene(prompt_method, supplement_version, player=player, max_turns=20)
            write_transcript(dialogue, (GAME + "/transcript_" + model_name + "_" +
                                        prompt_method + "_" + str(run_no) + ".txt"))

In [7]:
# # GENERATE ONE DIALOGUE

# player = False
# scene_version = 0
# supplement_version = -1

# (back_story, scene_description, scene_supplement,
#  opening_speech, queries) = load_prompts(scene_version, supplement_version)
# dialogue = sim_mini_scene(
#     back_story, scene_description, scene_supplement,
#     ACTORS, opening_speech, queries,
#     player=player, max_turns=20)